# Running simulations through the cloud 

This notebook is a tutorial of the API used for submitting simulations to our servers.

In [1]:
import tidy3d as td
import tidy3d.web as web

# set logging level to ERROR because we'll only running simulations to demonstrate the web API, we dont care about the results.
td.config.logging_level = "ERROR"


## Setup

Let's set up a simple simulation to get started.

In [2]:
# whether to print output in web functions, note: if False (default) they will all run silently
verbose = True

# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1 / fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]), medium=dielectric
)

# create source
source = td.UniformCurrentSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
    polarization="Ex",
)

# create monitor
monitor = td.FieldMonitor(
    fields=["Ex", "Ey", "Ez"],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="field",
)

# Initialize simulation
sim = td.Simulation(
    size=sim_size,
    grid_spec=td.GridSpec.uniform(dl),
    structures=[square],
    sources=[source],
    monitors=[monitor],
    run_time=run_time,
    boundary_spec=td.BoundarySpec.all_sides(boundary=pml),
)


## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
# upload the simulation to our servers
task_id = web.upload(sim, task_name="webAPI", verbose=verbose)

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id, verbose=verbose)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path="data/sim.hdf5", verbose=verbose)


[12:44:18] Created task 'webAPI' with task_id 'fdve-fecc0afb-0864-432b-a6d7-d2b4ff98c508v1'.          ]8;id=558727;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=558264;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[12:44:25] status = queued                                                                            ]8;id=948920;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=389791;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#269\269]8;;\

Output()

[12:44:31] status = preprocess                                                                        ]8;id=226910;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=207105;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#263\263]8;;\

...

[12:44:41] status = postprocess                                                                       ]8;id=29352;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=682622;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#331\331]8;;\

Output()

[12:44:44] status = success                                                                           ]8;id=979297;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=589863;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#338\338]8;;\

Output()

[12:44:49] loading SimulationData from data/sim.hdf5                                                  ]8;id=864094;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=742837;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the [web.run()](../_autosummary/tidy3d.web.run.html) function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to [web.start](../_autosummary/tidy3d.web.start.html) and then exit the session and load the results at a later time using [web.load](../_autosummary/tidy3d.web.load.html).


## Job Container

The [web.Job](../_autosummary/tidy3d.web.Job.html) interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original [Simulation](../_autosummary/tidy3d.Simulation.html).

While [Job](../_autosummary/tidy3d.web.Job.html) has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call `.run()` so we will do that now.

In [4]:
# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name="job", verbose=verbose)

# start job, monitor, and load results
sim_data = job.run(path="data/sim.hdf5")


[12:44:49] Created task 'job' with task_id 'fdve-b619ca0c-74b7-4152-9846-8506c3c5507dv1'.             ]8;id=412354;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=798824;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[12:44:51] status = queued                                                                            ]8;id=37711;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=829878;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#269\269]8;;\

Output()

[12:44:53] status = preprocess                                                                        ]8;id=158807;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=86130;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#263\263]8;;\

...

[12:45:07] status = postprocess                                                                       ]8;id=854276;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=64737;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#331\331]8;;\

Output()

[12:45:12] status = success                                                                           ]8;id=470594;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=628919;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#338\338]8;;\

Output()

[12:45:17] loading SimulationData from data/sim.hdf5                                                  ]8;id=608530;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=727336;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

Another convenient thing about [Job](../_autosummary/tidy3d.web.Job.html) objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [5]:
# saves the job metadata to a single file
job.to_file("data/job.json")

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file("data/job.json")

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path="data/sim.hdf5")


Output()

[12:45:18] loading SimulationData from data/sim.hdf5                                                  ]8;id=242447;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=872276;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in [Batch](../_autosummary/tidy3d.web.Batch.html) object.
The batch object is like a [Job](../_autosummary/tidy3d.web.Job.html), but stores task metadata for a series of simulations.

In [6]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f"sim_{i}": sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims, verbose=verbose)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir="data")


[12:45:18] Created task 'sim_0' with task_id 'fdve-cc019bf0-0b2d-46ab-8616-9b0698638c5av1'.           ]8;id=967450;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=963846;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[12:45:20] Created task 'sim_1' with task_id 'fdve-365989f1-9315-4aa2-90df-6e6553c6c5ccv1'.           ]8;id=412369;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=106456;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[12:45:21] Created task 'sim_2' with task_id 'fdve-5fabfe5c-ab16-40f5-91c8-bfd19b4c3166v1'.           ]8;id=45575;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=427560;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[12:45:28] Started working on Batch.                                                               ]8;id=886453;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=963352;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#402\402]8;;\

Output()

[12:45:53] Batch complete.                                                                         ]8;id=189462;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=883988;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#436\436]8;;\

When the batch is completed, the output is not a [SimulationData](../_autosummary/tidy3d.SimulationData.html) but rather a [BatchData](../_autosummary/tidy3d.web.BatchData.html).  The data within this [BatchData](../_autosummary/tidy3d.web.BatchData.html) object can either be indexed directly `batch_results[task_name]` or can be looped through `batch_results.items()` to get the [SimulationData](../_autosummary/tidy3d.SimulationData.html) for each task.

This was chosen to reduce the memory strain from loading all [SimulationData](../_autosummary/tidy3d.SimulationData.html) objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [7]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[12:45:58] loading SimulationData from data/fdve-cc019bf0-0b2d-46ab-8616-9b0698638c5av1.hdf5          ]8;id=96572;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=266397;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

Output()

[12:46:03] loading SimulationData from data/fdve-365989f1-9315-4aa2-90df-6e6553c6c5ccv1.hdf5          ]8;id=477088;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=441350;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

Output()

[12:46:05] loading SimulationData from data/fdve-5fabfe5c-ab16-40f5-91c8-bfd19b4c3166v1.hdf5          ]8;id=7422;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=263616;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

{'sim_0': 13540217.099674325, 'sim_1': 13540217.099674325, 'sim_2': 13540217.099674325}


## Asynchronous Batching

Finally, one can make use of the [asyncio package](https://realpython.com/async-io-python/) to perform asynchronous processing of several simulations.

For this purpose, a [web.run_async](../_autosummary/tidy3d.web.run_async.html) function is provided, which works like the regular `web.run` but accepts a dictionary of simulations. 

Here is the previous example repeated using this feature.

In [8]:
batch_results = web.run_async(simulations=sims, verbose=verbose)


Output()

In [9]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[12:46:39] loading SimulationData from ./fdve-df3d1b92-5e4d-4c22-84cc-9bf19bd7e6e9v1.hdf5             ]8;id=237702;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=624044;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

Output()

[12:46:41] loading SimulationData from ./fdve-820ab543-162f-4faa-b0a4-f4e6c023a356v1.hdf5             ]8;id=21631;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=211878;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

Output()

[12:46:45] loading SimulationData from ./fdve-f2e6b3b1-3149-40e3-86ea-d3d351865250v1.hdf5             ]8;id=617478;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=525989;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#510\510]8;;\

{'sim_0': 13540217.099674325, 'sim_1': 13540217.099674325, 'sim_2': 13540217.099674325}
